In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="eVeIuFmVXuAoqDnlp9kf")
project = rf.workspace("traffic-density-w6zd9").project("traffic-density-izacs-uhqed")
version = project.version(3)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Traffic-Density-3 in yolov8:: 100%|██████████| 33418/33418 [00:04<00:00, 6991.68it/s]


In [2]:
!pip install ultralytics
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 269.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 742.0 kB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from torch_geometric.nn import GCNConv

# Define the M2DNE module
class M2DNE(nn.Module):
    def __init__(self, in_channels, out_channels, num_layers=2):
        super().__init__()
        self.convs = nn.ModuleList([GCNConv(in_channels, out_channels) for _ in range(num_layers)])
    
    def forward(self, x, edge_indices_list):
        for i, (conv, edge_index) in enumerate(zip(self.convs, edge_indices_list)):
            x = torch.relu(conv(x, edge_index))
        return x

# Define the M2DNE-based Neck
class M2DNENeck(nn.Module):
    def __init__(self, backbone_channels=[256, 512, 1024], num_layers=2):
        super().__init__()
        self.m2dne_blocks = nn.ModuleList([
            M2DNE(backbone_channels[2], backbone_channels[2], num_layers),
            M2DNE(backbone_channels[1], backbone_channels[1], num_layers),
            M2DNE(backbone_channels[0], backbone_channels[0], num_layers)
        ])

    def forward(self, features, edge_indices_list):
        c3, c4, c5 = features
        p5 = self.m2dne_blocks[0](c5, edge_indices_list)
        p4 = self.m2dne_blocks[1](c4, edge_indices_list)
        p3 = self.m2dne_blocks[2](c3, edge_indices_list)
        return [p3, p4, p5]

# Modified YOLO Class with M2DNE Neck
class ModifiedYOLO(YOLO):
    def __init__(self, model_path='yolov8s.yaml'):
        super().__init__(model_path)
        self.replace_neck()
    
    def replace_neck(self):
        # Extract backbone channels
        backbone_channels = [
            self.model.model[6].cv2.conv.out_channels,
            self.model.model[8].cv2.conv.out_channels,
            self.model.model[9].cv1.conv.out_channels
        ]

        # Replace the neck with M2DNENeck
        for i, layer in enumerate(self.model.model):
            if isinstance(layer, nn.Sequential):  
                self.model.model[i] = M2DNENeck(backbone_channels=backbone_channels)
                break
    
    def forward(self, x, edge_indices_list):
        # Backbone
        features = self.model.model.backbone(x)  # Properly extract YOLOv8 backbone features

        # Neck (M2DNE)
        features = self.model.model.neck(features, edge_indices_list)  # Pass through M2DNE Neck
        
        # Head
        outputs = self.model.model.head(features)
        return outputs


# Training configuration
model = ModifiedYOLO('yolov8s.pt')

# Example usage
# You need to provide `edge_indices_list` (list of adjacency matrices) during training/inference
edge_indices_list = [torch.tensor([[0, 1], [1, 0]], dtype=torch.long)]  # Example adjacency matrix

model.train(
    data='/kaggle/working/Traffic-Density-3/data.yaml',
    epochs=50,
    batch=32,
    imgsz=640,
    lr0=0.01,
    lrf=0.1,
    optimizer="SGD",
    augment=True,
    mosaic=True,
    weight_decay=0.0005,
    dropout=0.5
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 182MB/s]


Ultralytics 8.3.75 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/Traffic-Density-3/data.yaml, epochs=50, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

100%|██████████| 755k/755k [00:00<00:00, 18.0MB/s]


Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:27<00:00,  1.06it/s]


                   all       1811       9377      0.775      0.789      0.832       0.56
               Bicycle        398        511      0.831       0.98      0.976      0.576
                E-bike        504        516      0.952       0.99      0.993      0.718
               Jeepney        837       1592      0.784      0.666      0.804      0.535
            Motorcycle        694       1556      0.658      0.632      0.664       0.37
            Pedestrian        568       1449      0.618      0.513      0.583      0.304
              Tricycle        684        882        0.9      0.809      0.925      0.665
                 Truck        728        882      0.838      0.889      0.919        0.7
               Vehicle        494       1989      0.618      0.835      0.791       0.61


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 10.9ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7cf86e727f40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,